In [2]:
import numpy as np
import pandas as pd
import time
import re
import os
import pandas_profiling
import random
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('../bank.csv', sep=';')

In [6]:
def generateMissingVal(df,ml):
    frac = ml
    idx = np.random.choice(range(df.shape[0]), int(df.count().sum()*frac), replace=True)
    cols = np.random.choice(range(df.shape[1]), size=len(idx), replace=True)
    x = df.astype(object).to_numpy()
    x[idx, cols] = np.nan
    df = pd.DataFrame(x, index=df.index, columns=df.columns)
    columns = df.columns
    c1 = ['job','marital','education','default','housing','loan','contact','month','poutcome','y']
    columns = list(set(columns)-set(c1))
    for col in columns:
        df[col] = pd.to_numeric(df[col])

    return df

        
def calcIQR(df):
    num = df._get_numeric_data()
    Q1 = num.quantile(0.25)
    Q3 = num.quantile(0.75)
    IQR = Q3 - Q1
    lW = Q1 - 1.5 * IQR
    uW = Q3 + 1.5 * IQR
    return lW, uW

def showOutliners(col,df,uW,lW,kind):
    return df[(df[col]>uW[col])|(df[col]<lW[col])]

def plotOutliners(col,df):
    sns.boxplot(data=df, y=col)
    df.boxplot(column=col)
    


In [7]:
#Test case

ml = 0.1
df = generateMissingVal(df,ml)
df.head()
lW,uW = calcIQR(df)
df.isna().sum()    
display(showOutliners('age',df,uW,lW,'both'))
# plotOutliners('age',df)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
36,78.0,retired,divorced,NaN,no,229.0,NaN,no,telephone,NaN,oct,97.0,1.0,-1.0,0.0,unknown,yes
129,77.0,retired,divorced,NaN,no,NaN,no,no,cellular,14.0,apr,161.0,1.0,-1.0,0.0,unknown,yes
166,78.0,housemaid,married,secondary,no,NaN,no,no,telephone,16.0,NaN,80.0,4.0,NaN,0.0,NaN,no
412,75.0,retired,divorced,tertiary,no,3810.0,NaN,no,cellular,16.0,nov,262.0,1.0,183.0,1.0,failure,yes
477,77.0,retired,married,tertiary,no,7802.0,no,no,telephone,NaN,may,421.0,1.0,92.0,2.0,success,NaN
573,81.0,retired,married,secondary,no,NaN,no,no,cellular,19.0,aug,65.0,5.0,-1.0,NaN,NaN,no
633,83.0,retired,NaN,secondary,no,0.0,no,NaN,cellular,18.0,mar,140.0,10.0,-1.0,0.0,unknown,no
688,80.0,management,married,primary,no,6483.0,no,no,telephone,19.0,oct,123.0,1.0,-1.0,0.0,NaN,no
1126,77.0,retired,married,secondary,no,610.0,no,no,cellular,13.0,aug,766.0,3.0,-1.0,0.0,unknown,no
1230,75.0,retired,divorced,secondary,no,1341.0,no,no,cellular,NaN,dec,415.0,7.0,-1.0,0.0,unknown,yes
